In [1]:
import requests,re,json
from urllib import request, parse, error
import socket
import logging
import ssl
from bs4 import BeautifulSoup
url = "https://www.bilibili.com/bangumi/play/ep339061"

In [2]:
def bilibili_headers(referer=None, cookie=None):
    # a reasonable UA
    ua = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36'
    headers = {'Accept': '*/*', 'Accept-Language': 'en-US,en;q=0.5', 'User-Agent': ua}
    if referer is not None:
        headers.update({'Referer': referer})
    if cookie is not None:
        headers.update({'Cookie': cookie})
    return headers
def bilibili_bangumi_api(avid, cid, ep_id, qn=0, fnval=16):
        return 'https://api.bilibili.com/pgc/player/web/playurl?avid=%s&cid=%s&qn=%s&type=&otype=json&ep_id=%s&fnver=0&fnval=%s' % (avid, cid, qn, ep_id, fnval)

In [3]:
def undeflate(data):
    """Decompresses data for Content-Encoding: deflate.
    (the zlib compression is used.)
    """
    import zlib
    decompressobj = zlib.decompressobj(-zlib.MAX_WBITS)
    return decompressobj.decompress(data)+decompressobj.flush()

def match1(text, *patterns):
    """Scans through a string for substrings matched some patterns (first-subgroups only).

    Args:
        text: A string to be scanned.
        patterns: Arbitrary number of regex patterns.

    Returns:
        When only one pattern is given, returns a string (None if no match found).
        When more than one pattern are given, returns a list of strings ([] if no match found).
    """

    if len(patterns) == 1:
        pattern = patterns[0]
        match = re.search(pattern, text)
        if match:
            return match.group(1)
        else:
            return None
    else:
        ret = []
        for pattern in patterns:
            match = re.search(pattern, text)
            if match:
                ret.append(match.group(1))
        return ret
def ungzip(data):
    """Decompresses data for Content-Encoding: gzip.
    """
    from io import BytesIO
    import gzip
    buffer = BytesIO(data)
    f = gzip.GzipFile(fileobj=buffer)
    return f.read()

def urlopen_with_retry(*args, **kwargs):
    retry_time = 3
    for i in range(retry_time):
        try:
            if True:
                # ignore ssl errors
                ctx = ssl.create_default_context()
                ctx.check_hostname = False
                ctx.verify_mode = ssl.CERT_NONE
                return request.urlopen(*args, context=ctx, **kwargs)
            else:
                return request.urlopen(*args, **kwargs)
        except socket.timeout as e:
            logging.debug('request attempt %s timeout' % str(i + 1))
            if i + 1 == retry_time:
                raise e
        # try to tackle youku CDN fails
        except error.HTTPError as http_error:
            logging.debug('HTTP Error with code{}'.format(http_error.code))
            if i + 1 == retry_time:
                raise http_error
                
def get_content(url, headers={}, decoded=True,cookies=""):
    """Gets the content of a URL via sending a HTTP GET request.

    Args:
        url: A URL.
        headers: Request headers used by the client.
        decoded: Whether decode the response body using UTF-8 or the charset specified in Content-Type.

    Returns:
        The content as a string.
    """

    logging.debug('get_content: %s' % url)

    req = request.Request(url, headers=headers)
    if cookies:
        cookies.add_cookie_header(req)
        req.headers.update(req.unredirected_hdrs)

    response = urlopen_with_retry(req)
    data = response.read()

    # Handle HTTP compression for gzip and deflate (zlib)
    content_encoding = response.getheader('Content-Encoding')
    if content_encoding == 'gzip':
        data = ungzip(data)
    elif content_encoding == 'deflate':
        data = undeflate(data)

    # Decode the response body
    if decoded:
        charset = match1(
            response.getheader('Content-Type', ''), r'charset=([\w-]+)'
        )
        if charset is not None:
            data = data.decode(charset, 'ignore')
        else:
            data = data.decode('utf-8', 'ignore')

    return data


In [4]:
html = get_content(url,headers =bilibili_headers())

In [5]:
pattern = r'__INITIAL_STATE__=(.*?);\(function\(\)'
text = html
def match1(text, *patterns):
    if len(patterns) == 1:
        pattern = patterns[0]
        match = re.search(pattern, text)
        if match:
            return match.group(1)
        else:
            return None
    else:
        ret = []
        for pattern in patterns:
            match = re.search(pattern, text)
            if match:
                ret.append(match.group(1))
        return ret

In [6]:
initial_state_text = match1(html, r'__INITIAL_STATE__=(.*?);\(function\(\)')  # FIXME
initial_state = json.loads(initial_state_text)


In [7]:
initial_state

{'loaded': True,
 'ver': {},
 'ssr': {},
 'loginInfo': {},
 'h1Title': '路人女主的养成方法 Fine：全片 ',
 'mediaInfo': {'stat': {'coins': 203598,
   'danmakus': 235111,
   'favorites': 4092353,
   'likes': 0,
   'reply': 28332,
   'share': 52215,
   'views': 6431681},
  'id': 28229860,
  'ssId': 34394,
  'title': '路人女主的养成方法 Fine',
  'jpTitle': '',
  'series': '路人女主的养成方法',
  'alias': '',
  'evaluate': '原作为丸户史明的轻小说《路人女主的养成方法》，本动画为该作TV动画的终章剧场版动画。高中生安艺伦也在樱花飞舞的坡道上与少女·加藤惠命运般地邂逅，并想着要制作以她为第一女主角的同人游戏。他邀请美术部所属、以同人插画师的身份活动的泽村·斯宾塞·英梨梨，以及学年第一的优等生、正在以轻小说作家身份活跃的霞之丘诗羽，共同成立了blessing software。并最终发表了第一部作品——\n之后，英梨梨和诗羽为了开发游戏大作《寰域编年纪》投靠了人气创作者红坂朱音。blessing software的代表伦也继续社团活动，和副代表惠一起开始开发新作。他起用学妹·波岛出海做插画师，并委托出海的哥哥·伊织当制作人，并与冰堂美智留及其乐队icy tail一同着手开发新作。\n英梨梨和诗羽的大作究竟如何？伦也和惠的关系将发生什么变化？blessing software的新作将去向何方？围绕路人女主的青春涂鸦，迎来大结局。',
  'ssType': 1,
  'ssTypeFormat': {'name': '番剧', 'homeLink': '//www.bilibili.com/anime/'},
  'status': 13,
  'multiMode': True,
  'forceWide': False,
  'specialCover': '',
  'squareCover': '//i0.hdsl

In [8]:
title = initial_state['h1Title']
ep_id = initial_state['epInfo']['id']
avid = initial_state['epInfo']['aid']
cid = initial_state['epInfo']['cid']
playinfos = []
api_url = bilibili_bangumi_api(avid, cid, ep_id)
print(api_url)
api_content = get_content(api_url, headers=bilibili_headers(referer=url))
api_playinfo = json.loads(api_content)
if api_playinfo['code'] == 0:  # success
    playinfos.append(api_playinfo)
current_quality = api_playinfo['result']['quality']
print(current_quality)
for fnval in [8, 16]:
    for qn in [120, 112, 80, 64, 32, 16]:
        # automatic format for durl: qn=0
        # for dash, qn does not matter
        if qn != current_quality:
            api_url = bilibili_bangumi_api(avid, cid, ep_id, qn=qn, fnval=fnval)
            api_content = get_content(api_url, headers=bilibili_headers(referer=url))
            api_playinfo = json.loads(api_content)
            if api_playinfo['code'] == 0:  # success
                playinfos.append(api_playinfo)

https://api.bilibili.com/pgc/player/web/playurl?avid=329652093&cid=237945449&qn=0&type=&otype=json&ep_id=339061&fnver=0&fnval=16
32


In [9]:
cid

237945449

In [10]:
for playinfo in playinfos:
    print(playinfo['result'])
    print(type(playinfo['result']))

{'accept_format': 'mp4,mp4,mp4,mp4,mp4', 'code': 0, 'durl': [{'size': 28845913, 'ahead': '', 'length': 360109, 'vhead': '', 'backup_url': ['https://upos-sz-mirrorkodo.bilivideo.com/upgcxcode/49/54/237945449/237945449-1-416.mp4?e=ig8euxZM2rNcNbNVhwdVhoMghwdVhwdEto8g5X10ugNcXBlqNxHxNEVE5XREto8KqJZHUa6m5J0SqE85tZvEuENvNo8g2ENvNo8i8o859r1qXg8xNEVE5XREto8GuFGv2U7SuxI72X6fTr859r1qXg8gNEVE5XREto8z5JZC2X2gkX5L5F1eTX1jkXlsTXHeux_f2o859IB_&uipk=5&nbs=1&deadline=1603529792&gen=playurl&os=kodobv&oi=3071155713&trid=b88cb513ec0647739c565d52d716c056p&platform=pc&upsig=3e5845e51eb193a57e147c47d2e5fe9d&uparams=e,uipk,nbs,deadline,gen,os,oi,trid,platform&mid=0&orderid=1,3&agrr=0&logo=40000000', 'https://upos-sz-mirrorkodob.bilivideo.com/upgcxcode/49/54/237945449/237945449-1-416.mp4?e=ig8euxZM2rNcNbNVhwdVhoMghwdVhwdEto8g5X10ugNcXBlqNxHxNEVE5XREto8KqJZHUa6m5J0SqE85tZvEuENvNo8g2ENvNo8i8o859r1qXg8xNEVE5XREto8GuFGv2U7SuxI72X6fTr859r1qXg8gNEVE5XREto8z5JZC2X2gkX5L5F1eTX1jkXlsTXHeux_f2o859IB_&uipk=5&nbs=1&deadl

In [11]:
len(playinfos)

11

In [ ]:
for playinfo in playinfos:
    if 'durl' in playinfo['result']:
        for video in playinfo['result']['durl']:
            baseurl = video['backup_url']
            print(baseurl[0])
            print(baseurl[1])
            htm = get_content(baseurl[0], headers=bilibili_headers(referer=url))

In [ ]:
def bilibili_download_by_cids(cids, title, output_dir='.', merge=True, info_only=False):
    urls = []
    for cid in cids:
        sign_this = hashlib.md5(bytes('appkey=' + appkey + '&cid=' + cid + secretkey, 'utf-8')).hexdigest()
        url = 'http://interface.bilibili.com/playurl?appkey=' + appkey + '&cid=' + cid + '&sign=' + sign_this
        urls += [i
                if not re.match(r'.*\.qqvideo\.tc\.qq\.com', i)
                else re.sub(r'.*\.qqvideo\.tc\.qq\.com', 'http://vsrc.store.qq.com', i)
                for i in parse_cid_playurl(get_content(url, headers=client))]

    type_ = ''
    size = 0
    for url in urls:
        _, type_, temp = url_info(url)
        size += temp

    print_info(site_info, title, type_, size)
    if not info_only:
        download_urls(urls, title, type_, total_size=None, output_dir=output_dir, merge=merge)

In [ ]:
#hash
import hashlib
def bilibili_interface_api(cid, qn=0):
    entropy = 'rbMCKn@KuamXWlPMoJGsKcbiJKUfkPF_8dABscJntvqhRSETg'
    appkey, sec = ''.join([chr(ord(i) + 2) for i in entropy[::-1]]).split(':')
    params = 'appkey=%s&cid=%s&otype=json&qn=%s&quality=%s&type=' % (appkey, cid, qn, qn)
    chksum = hashlib.md5(bytes(params + sec, 'utf8')).hexdigest()
    return 'https://interface.bilibili.com/v2/playurl?%s&sign=%s' % (params, chksum)

appkey='85eb6835b0a1034e';
secretkey = '2ad42749773c441109bdc0191257a664'
client = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Charset': 'UTF-8,*;q=0.5',
    'Accept-Encoding': 'gzip,deflate,sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    #'User-Agent': 'Biligrab /0.8 (cnbeining@gmail.com)'
    'User-Agent': "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/40.0.2214.94 Safari/537.36"
}
sign_this = hashlib.md5(bytes('appkey=' + appkey + '&cid=' + str(cid) + secretkey, 'utf-8')).hexdigest()
url = 'http://interface.bilibili.com/playurl?appkey=' + appkey + '&cid=' + str(cid) + '&sign=' + sign_this

In [15]:

url

'http://interface.bilibili.com/playurl?appkey=85eb6835b0a1034e&cid=237945449&sign=28da1dfc018910389b45595a5686a8d5'

In [ ]:
#下载 函数
def url_save(
    url, filepath, bar, refer=None, is_part=False, faker=False,
    headers=None, timeout=None, **kwargs):
    tmp_headers = headers.copy() if headers is not None else {}
    # When a referer specified with param refer,
    # the key must be 'Referer' for the hack here
    if refer is not None:
        tmp_headers['Referer'] = refer
    if type(url) is list:
        chunk_sizes = [url_size(url, faker=faker, headers=tmp_headers) for url in url]
        file_size = sum(chunk_sizes)
        is_chunked, urls = True, url
    else:
        file_size = url_size(url, faker=faker, headers=tmp_headers)
        chunk_sizes = [file_size]
        is_chunked, urls = False, [url]

    continue_renameing = True
    while continue_renameing:
        continue_renameing = False
        if os.path.exists(filepath):
            if not force and (file_size == os.path.getsize(filepath) or skip_existing_file_size_check):
                if not is_part:
                    if bar:
                        bar.done()
                    if skip_existing_file_size_check:
                        log.w(
                            'Skipping {} without checking size: file already exists'.format(
                                tr(os.path.basename(filepath))
                            )
                        )
                    else:
                        log.w(
                            'Skipping {}: file already exists'.format(
                                tr(os.path.basename(filepath))
                            )
                        )
                else:
                    if bar:
                        bar.update_received(file_size)
                return
            else:
                if not is_part:
                    if bar:
                        bar.done()
                    if not force and auto_rename:
                        path, ext = os.path.basename(filepath).rsplit('.', 1)
                        finder = re.compile(' \([1-9]\d*?\)$')
                        if (finder.search(path) is None):
                            thisfile = path + ' (1).' + ext
                        else:
                            def numreturn(a):
                                return ' (' + str(int(a.group()[2:-1]) + 1) + ').'
                            thisfile = finder.sub(numreturn, path) + ext
                        filepath = os.path.join(os.path.dirname(filepath), thisfile)
                        print('Changing name to %s' % tr(os.path.basename(filepath)), '...')
                        continue_renameing = True
                        continue
                    if log.yes_or_no('File with this name already exists. Overwrite?'):
                        log.w('Overwriting %s ...' % tr(os.path.basename(filepath)))
                    else:
                        return
        elif not os.path.exists(os.path.dirname(filepath)):
            os.mkdir(os.path.dirname(filepath))

    temp_filepath = filepath + '.download' if file_size != float('inf') \
        else filepath
    received = 0
    if not force:
        open_mode = 'ab'

        if os.path.exists(temp_filepath):
            received += os.path.getsize(temp_filepath)
            if bar:
                bar.update_received(os.path.getsize(temp_filepath))
    else:
        open_mode = 'wb'

    chunk_start = 0
    chunk_end = 0
    for i, url in enumerate(urls):
        received_chunk = 0
        chunk_start += 0 if i == 0 else chunk_sizes[i - 1]
        chunk_end += chunk_sizes[i]
        if received < file_size and received < chunk_end:
            if faker:
                tmp_headers = fake_headers
            '''
            if parameter headers passed in, we have it copied as tmp_header
            elif headers:
                headers = headers
            else:
                headers = {}
            '''
            if received:
                # chunk_start will always be 0 if not chunked
                tmp_headers['Range'] = 'bytes=' + str(received - chunk_start) + '-'
            if refer:
                tmp_headers['Referer'] = refer

            if timeout:
                response = urlopen_with_retry(
                    request.Request(url, headers=tmp_headers), timeout=timeout
                )
            else:
                response = urlopen_with_retry(
                    request.Request(url, headers=tmp_headers)
                )
            try:
                range_start = int(
                    response.headers[
                        'content-range'
                    ][6:].split('/')[0].split('-')[0]
                )
                end_length = int(
                    response.headers['content-range'][6:].split('/')[1]
                )
                range_length = end_length - range_start
            except:
                content_length = response.headers['content-length']
                range_length = int(content_length) if content_length is not None \
                    else float('inf')

            if is_chunked:  # always append if chunked
                open_mode = 'ab'
            elif file_size != received + range_length:  # is it ever necessary?
                received = 0
                if bar:
                    bar.received = 0
                open_mode = 'wb'

            with open(temp_filepath, open_mode) as output:
                while True:
                    buffer = None
                    try:
                        buffer = response.read(1024 * 256)
                    except socket.timeout:
                        pass
                    if not buffer:
                        if is_chunked and received_chunk == range_length:
                            break
                        elif not is_chunked and received == file_size:  # Download finished
                            break
                        # Unexpected termination. Retry request
                        tmp_headers['Range'] = 'bytes=' + str(received - chunk_start) + '-'
                        response = urlopen_with_retry(
                            request.Request(url, headers=tmp_headers)
                        )
                        continue
                    output.write(buffer)
                    received += len(buffer)
                    received_chunk += len(buffer)
                    if bar:
                        bar.update_received(len(buffer))

    assert received == os.path.getsize(temp_filepath), '%s == %s == %s' % (
        received, os.path.getsize(temp_filepath), temp_filepath
    )

    if os.access(filepath, os.W_OK):
        # on Windows rename could fail if destination filepath exists
        os.remove(filepath)
    os.rename(temp_filepath, filepath)